In [13]:
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from numpy import linalg as LA
from tqdm import tqdm
from mpl_toolkits import mplot3d
import matplotlib.patches as mpatches
import umap

%matplotlib qt

C:\Users\bscuser\anaconda3\envs\pytorch_medulloblastoma\lib\site-packages\umap\__init__.py:9: UserWarning: Tensorflow not installed; ParametricUMAP will be unavailable
  warn("Tensorflow not installed; ParametricUMAP will be unavailable")


In [14]:
def distances_generated(data,colors):
    
    if type(data) == torch.tensor:
        data = data.numpy()
    
    minor_list = []
    index_list = []

    for i in range(0,len(data)):
        minor = 9999
        index = 0
        for j in range(0,len(data)):
            if i != j:
                a=LA.norm(data[i]-data[j],ord=1)
                if colors[j] in ['Generated','Reference1','Reference2']: 
                    a = 9999
                if a < minor:
                    minor = a
                    index = j
        print(i, minor, index, colors[i], colors[index])
        minor_list.append(minor)
        if colors[i] == 'Generated':
            index_list.append(index)
  #  print("Average nearest distance w/o Generated:", np.cumsum(minor_list[:285])[-1]/len(minor_list[:285]))
   # print("Average nearest distance Generated:", np.cumsum(minor_list[285:])[-1]/len(minor_list[285:]))

    return minor_list, set(index_list)

In [15]:
def metapatients(data, subgroups):
    mean_df = pd.DataFrame(data) #mean = coded
    mean_df['Subgroup'] = subgroups[1] #add subgroups column to the embedding data

    mean_shh = mean_df[mean_df['Subgroup'] == 'SHH']
    mean_g3 = mean_df[mean_df['Subgroup'] == 'Group3']
    mean_g4 = mean_df[mean_df['Subgroup'] == 'Group4']
    #mean_wnt = mean_df[mean_df['Subgroup'] == 'WNT']

    # Centroids of Means
    standard_shh = mean_shh.mean().values.reshape(1,len(data[0])) 
    standard_g3 = mean_g3.mean().values.reshape(1,len(data[0]))
    standard_g4 = mean_g4.mean().values.reshape(1,len(data[0]))
    #standard_wnt = mean_wnt.mean().values.reshape(1,len(data[0]))

    return standard_g3, standard_g4, standard_shh#, standard_wnt


In [16]:
def umap_plot(data,color,components=2):
    
    #n_neighbors = [5,15,22,30]
    n_neighbors = [15]
    

    for i in n_neighbors:
        reducer = umap.UMAP(n_components=components,n_neighbors=i, densmap=True)
        embedding = reducer.fit_transform(data)
        embedding_df = pd.DataFrame(embedding)
        embedding_df['Subgroups']= color

        X_data= embedding_df[0]
        Y_data = embedding_df[1]
        if components == 3:
            Z_data = embedding_df[2]
        Sbgrp = embedding_df['Subgroups']

        cdict = {'Group4': 'pink', 'SHH': 'blue', 'WNT': 'green', 'Group3': 'gold', 'Generated': 'black', 'Reference1': 'red', 'Reference2': 'red'}
        c = [cdict[val] for val in Sbgrp]

        plt.figure(figsize=(16,10))
        if components == 3:
            ax = plt.axes(projection='3d')
            ax.scatter3D(X_data, Y_data, Z_data, c=c)
        if components == 2:
            plt.scatter(X_data,Y_data,c=c)
        pink_c = mpatches.Patch(color='pink', label='Group4')
        blue_c = mpatches.Patch(color='blue', label='SHH')
        green_c = mpatches.Patch(color='green', label='WNT')
        yellow_c = mpatches.Patch(color='gold', label='Group3')
        black_c = mpatches.Patch(color='black', label='Generated')
        red_c = mpatches.Patch(color='red', label='Reference')
        plt.legend(handles=[pink_c,blue_c,green_c,yellow_c,black_c,red_c])
        plt.title('UMAP with n_neighbors %i'%(i))
        plt.show()
    

In [17]:
def get_embeddings(model,dataloader):
    model.eval()
    rec_model = np.zeros(shape=(0,model.decoder[2].out_features))
    embedding_model = np.zeros(shape=(0,features))
    mean_model = np.zeros(shape=(0,features))
    logvar_model = np.zeros(shape=(0,features))
    with torch.no_grad(): # in validation we don't want to update weights
        for i, data in tqdm(enumerate(dataloader), total=int(len(test_dataset)/dataloader.batch_size)):
            data = data.to(device)
            data = data.view(data.size(0), -1)
            reconstruction,mean,logvar, coded = model(data)
            rec_model = np.concatenate((rec_model, reconstruction), axis=0)
            mean_model = np.concatenate((mean_model, mean), axis=0)
            logvar_model = np.concatenate((logvar_model, logvar), axis=0)
            embedding_model = np.concatenate((embedding_model,coded),axis=0)
    return rec_model, embedding_model, mean_model, logvar_model

In [18]:
def data_generation(N,subgroup,test_dataset):

  #  if subgroup == 'G4':
  #      data = meta_g4
  #  elif subgroup == 'SHH':
  #      data = meta_shh
  #  elif subgroup == 'G3':
  #      data = meta_g3
  #  else:
  #      print("Incorrect subgroup")
  #      return    

    data = test_dataset[subgroup]
    
    sample = np.zeros(shape=(1,features))

  #  data = torch.from_numpy(data).float()

    with torch.no_grad():
        data = data.to(device)
        reconstruction, mean, logvar, coded = model(data)          

    for i in range(0,N):
        std = torch.exp(0.5*logvar) 
        eps = torch.randn_like(std) 
        resultado = mean + (eps*std)
        sample = np.concatenate((sample, resultado), axis=0)

    sample = sample.reshape(N+1,features)
    z = sample[1:]

    z = torch.from_numpy(z)
    z = z.float()

    with torch.no_grad():  
        z = z.to(device)
        samples = model.decoder(z)   #decode the data
    generated = torch.cat([test_dataset, samples], dim=0) #concat the test data and the generate data to visualize it
    new_colors = np.array(['Generated']*len(samples)) #create the reference to paint black the generate examples
    colors_generated = np.concatenate((colors,new_colors),axis=0) #concat the colors of the test data and the generate data

    return generated, colors_generated

In [19]:
def data_interpolation(N,patient1,patient2, colors, test_dataset,logvar_true = True):

    z1 = torch.from_numpy(patient1).float() #Means
    z2 = torch.from_numpy(patient2).float() #Logvar

    with torch.no_grad(): 
        z1 = z1.to(device)
        z2 = z2.to(device)
        reference1, mean1, logvar1, _ = model(z1)                
        reference2, mean2, logvar2, _ = model(z2)  

    sample = np.zeros(shape=(1,features))
    for i in range(0,N):
        mean = i / (N - 1) * mean2 + (1 - i / (N - 1) ) * mean1 #interpolation mean
        if logvar_true == True:
            std1 = torch.exp(0.5*logvar1)
            std2 = torch.exp(0.5*logvar2)
            std = i / (N - 1) * std2 + (1 - i / (N - 1) ) * std1 #interpolation logvar
        else:
            std = 0
        eps = torch.randn_like(std) 
        resultado = mean + (eps*std)
        sample = np.concatenate((sample, resultado), axis=0)
    sample = sample.reshape(N+1,features)
    z = sample[1:]

    z = torch.from_numpy(z) #preprocessing to introduce samples in NN
    z = z.float()

    
    #GENERATE INTERPOLATION DATA
    with torch.no_grad():        
        z = z.to(device)
        samples = model.decoder(z)   #decode the data
    generated = torch.cat([test_dataset, samples], dim=0) #concat the test data and the generate data to visualize it
    new_colors = np.array(['Generated']*len(samples)) #create the reference to paint black the generate examples
    colors_generated = np.concatenate((colors,new_colors),axis=0) #concat the colors of the test data and the generate data

    # ADD REFERENCES
    generated = torch.cat((generated,reference1),axis=0) # add the centroids to the data to plot them
    colors_reference = np.array(['Reference1']) #add label generated
    colors_generated = np.concatenate((colors_generated,colors_reference),axis=0) #generate new colors

    generated = torch.cat((generated,reference2),axis=0) # add the centroids to the data to plot them
    colors_reference = np.array(['Reference2']) #add label generated
    colors_generated = np.concatenate((colors_generated,colors_reference),axis=0) #generate new colors

    return generated, colors_generated

In [20]:
def data_interpolation_generation(N,patient1,patient2, colors, test_dataset,logvar_true = True):

    z1 = torch.from_numpy(patient1).float() #Means
    z2 = torch.from_numpy(patient2).float() #Logvar

    with torch.no_grad():
        z1 = z1.to(device)
        z2 = z2.to(device)
        reference1, mean1, logvar1, _ = model(z1)                
        reference2, mean2, logvar2, _ = model(z2)  

    sample = np.zeros(shape=(1,features))
    for i in range(0,N):
        mean = i / (N - 1) * mean2 + (1 - i / (N - 1) ) * mean1 #interpolation mean
        if logvar_true == True:
            std1 = torch.exp(0.5*logvar1)
            std2 = torch.exp(0.5*logvar2)
            std = i / (N - 1) * std2 + (1 - i / (N - 1) ) * std1 #interpolation logvar
        else:
            std = 0
        eps = torch.randn_like(std) 
        resultado = mean + (eps*std)
        sample = np.concatenate((sample, resultado), axis=0)
    sample = sample.reshape(N+1,features)
    z = sample[1:]

    z = torch.from_numpy(z) #preprocessing to introduce samples in NN
    z = z.float()

    
    #GENERATE INTERPOLATION DATA
    with torch.no_grad():      
        z = z.to(device)
        samples = model.decoder(z)   #decode the data
        
    return samples

### Train Data

In [21]:
data = pd.read_csv('Medulloblastoma Files\Medulloblastoma_Cavalli_VAE_data_Less.csv', sep=',', na_values=".")
print("The shape of the data is: ", data.shape)
data = data.rename(columns={'Unnamed: 0': 'Patient'})

subgroups = pd.read_csv('Medulloblastoma Files\GSE85218_subgroups.csv', sep=' ',header=None)
print("The shape of the subgroups is: ", subgroups.shape)
colors_train = subgroups[1].values

The shape of the data is:  (763, 5669)
The shape of the subgroups is:  (763, 2)


### Test data

In [22]:
data_test = pd.read_csv('Medulloblastoma Files\Medulloblastoma_Northcott_VAE_data_Less.csv', sep=',', na_values=".")
print("The shape of the data is: ", data_test.shape)
data_test = data_test.rename(columns={'Unnamed: 0': 'Patient'})

subgroups_test = pd.read_csv('Medulloblastoma Files\GSE37382_subgroups.csv', sep=' ',header=None)
print("The shape of the subgroups is: ", subgroups_test.shape)
colors = subgroups_test[1].values

The shape of the data is:  (285, 5669)
The shape of the subgroups is:  (285, 2)


In [23]:
data = data.drop(['Patient'],axis=1)
data_test = data_test.drop(['Patient'],axis=1)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

scaler.fit(data)
data = scaler.transform(data) #(x - mu / s) almost all values between -1,1

scaler.fit(data_test)
data_test = scaler.transform(data_test)

In [24]:
data = data.drop(['Patient'],axis=1)
data_test = data_test.drop(['Patient'],axis=1)

from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

scaler.fit(data)
data = scaler.transform(data) #(x - mu / s) almost all values between -1,1

scaler.fit(data_test)
data_test = scaler.transform(data_test)

AttributeError: 'numpy.ndarray' object has no attribute 'drop'

In [ ]:
umap_plot(data_test,colors,components=2)

In [ ]:
data = pd.DataFrame(data)
train_dataset = torch.tensor(data.values).float()

data_test = pd.DataFrame(data_test)
test_dataset = torch.tensor(data_test.values).float()


train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=8,
    shuffle=True,
)

test_loader = torch.utils.data.DataLoader(
    test_dataset,
    batch_size=8,
    shuffle=False,
)

In [25]:
features = 32

class VAE(nn.Module):
    def __init__(self, **kwargs):
        super().__init__()

        self.encoder = nn.Sequential(
            nn.Linear(in_features=kwargs["input_shape"], out_features=kwargs["mid_dim"]),
            nn.ReLU(),
            nn.Linear(in_features=kwargs["mid_dim"], out_features=features*2)
        )
        
        self.decoder = nn.Sequential(
            nn.Linear(in_features=features, out_features=kwargs["mid_dim"]),
            nn.ReLU(),
            nn.Linear(in_features=kwargs["mid_dim"], out_features=kwargs["input_shape"]),
            nn.Tanh()
            #nn.Sigmoid()
        )

    def reparametrize(self, mu, log_var):

        if self.training:
            std = torch.exp(0.5*log_var) 
            eps = torch.randn_like(std) 
            sample = mu + (eps*std) 
        else:
            sample = mu
        return sample

    def forward(self, x):
        
        mu_logvar = self.encoder(x).view(-1,2,features)
        mu = mu_logvar[:, 0, :] 
        log_var = mu_logvar[:, 1, :] 

        z = self.reparametrize(mu,log_var) 
        reconstruction = self.decoder(z)
        
        return reconstruction, mu, log_var, z
    

#model = VAE(input_shape=5668, mid_dim=2048)
#criterion = nn.BCELoss(reduction='sum')
#criterion = nn.MSELoss()
#optimizer = optim.Adam(model.parameters(), lr=0.0001)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


C:\Users\bscuser\anaconda3\envs\pytorch_medulloblastoma\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [ ]:
def final_loss(mu, logvar, reconstruction_loss, beta):
    HALF_LOG_TWO_PI = 0.91893
    beta = 0.000001
    KL_divergence = beta * 0.5 * torch.sum(logvar.exp() - logvar - 1 + mu.pow(2)) 
    gamma = torch.sqrt(reconstruction_loss)
    log_gamma = torch.log(gamma)
    Reconstruction = reconstruction_loss/(2*gamma) + log_gamma + HALF_LOG_TWO_PI

    return KL_divergence + Reconstruction#, _

In [ ]:
def fit(model, dataloader, beta):
    model.train() #network in train mode
    running_loss = 0.0
    gamma_loss = 0.0
    for i, data in tqdm(enumerate(dataloader), total=int(len(train_dataset)/dataloader.batch_size)):
        data = data # we want the data, not the label
        data = data.view(data.size(0), -1) #flat the data
        optimizer.zero_grad() # reset the gradients back to zero
        reconstruction, mu, logvar,_ = model(data)  # compute reconstructions
        reconstruction_loss = criterion(reconstruction, data) #calculate reconstruction loss
        loss = final_loss(mu, logvar, reconstruction_loss, beta)# real loss: reconstruction + kl_divergence
        running_loss += loss.item() 
        loss.backward() # compute accumulated gradients
        optimizer.step() #update the weights (net.parameters)
    train_loss = running_loss/len(dataloader.dataset) # average loss
    return train_loss

In [ ]:
def validate(model, dataloader, beta):
    model.eval() #network in evaluation mode
    running_loss = 0.0
    with torch.no_grad(): # in validation we don't want to update weights
        for i, data in tqdm(enumerate(dataloader), total=int(len(test_dataset)/dataloader.batch_size)):
            data = data
            data = data.view(data.size(0), -1)
            reconstruction, mu, logvar, coded = model(data)
            reconstruction_loss = criterion(reconstruction, data)
            loss  = final_loss(mu, logvar, reconstruction_loss, beta)
            running_loss += loss.item()
            
    val_loss = running_loss/len(dataloader.dataset)
    return val_loss

In [ ]:
epochs = 200  #the loss stuck up at this epoch
batch_size = 16

train_loss = []
test_loss = []
for epoch in range(epochs):
    print(f"\n Epoch {epoch+1} of {epochs}")
    if epoch < 10:
        beta_launcher = 0
    elif 20 <= epoch <= 40:
        beta_launcher = (0.05*epoch - 1)
    elif epoch > 40:
        beta_launcher = 1
    test_epoch_loss = validate(model, test_loader, beta_launcher)
    train_epoch_loss = fit(model, train_loader, beta_launcher)
    train_loss.append(train_epoch_loss)
    test_loss.append(test_epoch_loss)
    print(f"\nTrain Loss: {train_epoch_loss:.4f}")
    print(f"Test Loss: {test_epoch_loss:.4f}")
    print(f"Betha value: {beta_launcher: 4f}")
plot_train_test(train_loss,test_loss)

In [ ]:
PATH = './vaeAnnealing32MSE.pth'
torch.save(model.state_dict(), PATH) #save in a dictionary all paramete

In [26]:
PATH = './vaeAnnealing32MSE.pth'
model = VAE(input_shape=5668, mid_dim=2048).to(device)
model.load_state_dict(torch.load(PATH))

<All keys matched successfully>

In [ ]:
reconstructed, coded, mean, logvar = get_embeddings(model, test_loader)

In [ ]:
meta_g3, meta_g4, meta_shh = metapatients(reconstructed, subgroups_test)

In [ ]:
M_distance_g3_shh = LA.norm(meta_shh - meta_g3,ord=1)
M_distance_G3G4 = LA.norm(meta_g4 - meta_g3,ord=1)
M_distance_shh_G4 = LA.norm(meta_g4 - meta_shh,ord=1)

print("Manhattan Distance G3-SHH: ",M_distance_g3_shh)
print("Manhattan Distance G3-G4: ",M_distance_G3G4)
print("Manhattan Distance SHH-G4: ",M_distance_shh_G4)

E_distance_g3_shh = LA.norm(meta_shh - meta_g3,ord=2)
E_distance_G3G4 = LA.norm(meta_g4 - meta_g3,ord=2)
E_distance_shh_G4 = LA.norm(meta_g4 - meta_shh,ord=2)

print("\nEuclidean Distance G3-SHH: ",E_distance_g3_shh)
print("Euclidean Distance G3-G4: ",E_distance_G3G4)
print("Euclidean Distance SHH-G4: ",E_distance_shh_G4)

In [ ]:
fig = umap_plot(mean,colors)

In [ ]:
generated, new_colors = data_generation(30, 80, test_dataset) #generate 64 data of G3

In [ ]:
umap_plot(generated,new_colors) #plot it in UMAP with 2 dimensions
#umap_plot(generated,new_colors,3) plot it in UMAP with 3 dimensions

In [ ]:
interpolation_data, interpolation_colors = data_interpolation(32,test_dataset[95].detach().numpy(),test_dataset[200].detach().numpy(),colors,test_dataset, True)

In [ ]:
umap_plot(interpolation_data,interpolation_colors,3) #plot it in UMAP with 2 dimensions

In [ ]:
interpolation_data, interpolation_colors = data_interpolation(32,meta_g3,meta_g4,colors,test_dataset, True)

In [ ]:
minor, near_generated = distances_generated(interpolation_data,interpolation_colors)

In [ ]:
candidates = []

for i in range(0,len(test_dataset)):
    for j in range(0,len(test_dataset)):
        if colors[i] != colors[j]:
            interpolation_data, interpolation_colors = data_interpolation(32,test_dataset[i].detach().numpy(),test_dataset[j].detach().numpy(),colors,test_dataset, True)
            minor, near_generated = distances_generated(interpolation_data,interpolation_colors)
            if len(near_generated) >= 10:
                print("Candidates: ", i, " --- ",j)
            else:
                print("Nothing in ", i, " --- ", j)


In [ ]:
interpolation_data, interpolation_colors = data_interpolation(32,test_dataset[2].detach().numpy(),test_dataset[175].detach().numpy(),colors,test_dataset, True)
minor, near_generated = distances_generated(interpolation_data,interpolation_colors)

In [ ]:
near_generated

In [ ]:
sample = np.zeros(shape=(features,))

N = 200
z = torch.randn((N, features))
with torch.no_grad():
    sample = model.decoder(z)

generated = torch.cat([train_dataset, sample], dim=0) #concat the test data and the generate data to visualize it
new_colors = np.array(['Generated']*len(sample)) #create the reference to paint black the generate examples
colors_generated = np.concatenate((colors_train,new_colors),axis=0) #concat the colors of the test data and the generate data

In [ ]:
umap_plot(generated,colors_generated)

In [ ]:
generated = torch.cat([test_dataset, torch.from_numpy(meta_g3).float(), torch.from_numpy(meta_g4).float(), torch.from_numpy(meta_shh).float()], dim=0) #concat the test data and the generate data to visualize it
new_colors = np.array(['Generated']*len(meta_g3)*3) #create the reference to paint black the generate examples
colors_generated = np.concatenate((colors,new_colors),axis=0) #concat the colors of the test data and the generate data